### Cosmic-Ray modified Spectrum

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def model(x, t, a, rsub, rtot):
    p     = t
    u     = x[0]
    dudp  = x[1]
    
    coef  = -(gamma+1)*u**(-(gamma+2))/(m0**2-u**(-(gamma+1)))*dudp
    vp    = 1./np.sqrt(1.+1./p**2)
    dvp   = (1+1./p**2)**(-1.5)/p**3
    coef += 3/p +dvp/vp
    coef += -rtot/(rtot*u-1)*dudp - 3*rtot*u/p/(rtot*u-1)
    
    dudp2 = dudp*coef 
    return [dudp,dudp2]

In [3]:
'''
Calculate CR modified shock spectrum
'''
def CR_modified_spectrum(m0,eta,c_u0,logpinj,logpmax,u1min,u1max, num=200,error=0.01,gamma=5./3):
    sol = 0
    for i in range(num):
        a    = (u1max-u1min)/num*i+u1min # a = u1/u0
        m1   = m0*a**((gamma + 1.)/2.);
        rsub = (gamma + 1.)*m1**2/((gamma - 1.)*m1**2 + 2.) # compression ratio for sub-shock
        qs   = 3.*rsub/(rsub - 1.)
        # total compression ratio
        rtot = m0**(2./(gamma+1))*(((gamma+1)*rsub**gamma-(gamma-1)*rsub**(gamma+1.))/2.)**(1./(gamma + 1.)) 
        if rsub<1. or rsub>4:
            print('Error in compression ratio: Rsub!')
            continue

        p = np.logspace(logpinj,logpmax,200)
        # initial condition
        u0    = a
        dudp0 = 1./3*eta*qs*rtot/rsub/np.sqrt(1.+1./p[0]**2)*c_u0**2/((rtot*a-1)/(rsub-1))/(1.-a**(-(gamma+1))/m0**2)

        up = odeint(model,[u0,dudp0],p,args=(a, rsub, rtot))
        if up[-1,0]>1.-error and up[-1,0]<1+error:
            sol = 1
            print ('Solution found!')
            break
    if sol==0:
        print ('No solution satisfies input parameters!')
    ### calculate spectrum
    Qp = np.linspace(0,1,len(up))
    for i in range(len(up)):
        if i==0:
            dup = (up[i+1,0]-up[i,0])/(p[i+1]-p[i])
            print dup, (up[i+1,0]-up[i,0]), (p[i+1]-p[i])
        elif i==len(up)-1:
            dup = (up[i,0]-up[i-1,0])/(p[i]-p[i-1])
        else:
            dup = (up[i+1,0]-up[i-1,0])/(p[i+1]-p[i-1])
        
        Qp[i] = -3*up[i,0]/(up[i,0]-1/rtot) - p[i]/(up[i,0]-1/rtot)*dup
        
    # x: p/mc, y: p^4 f(p)
    return p, p**(Qp+4.)

In [4]:
def test():
    mach    = 43    # sonic mach number
    eta     = 1e-3  # injection efficiency
    c_u0    = 60    # c/u0
    gamma   = 5/3.0 # adiabatic index
    error   = 0.5   # how far final u1/u0 from 1
    logpinj = -2    # log min momentum
    logpmax = 5     # log max momentum 
    u1min   = 0.1   #u1/u0 minimal value
    u1max   = 0.9   # u1/u0 max value
    num     = 2000  # iteration num for u1 from min to max
    p,fp = CR_modified_spectrum(mach,eta,c_u0,logpinj,logpmax,u1min,u1max, num,error,gamma)
    plt.loglog(p,fp)
    plt.xlabel('p/mc')
    plt.ylabel('$p^4f(p)$')